In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
database='database.sqlite'

In [3]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

type               name           tbl_name  rootpage  \
0  table    sqlite_sequence    sqlite_sequence         4   
1  table  Player_Attributes  Player_Attributes        11   
2  table             Player             Player        14   
3  table              Match              Match        18   
4  table             League             League        24   
5  table            Country            Country        26   
6  table               Team               Team        29   
7  table    Team_Attributes    Team_Attributes         2   

                                                 sql  
0             CREATE TABLE sqlite_sequence(name,seq)  
1  CREATE TABLE "Player_Attributes" (\n\t`id`\tIN...  
2  CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...  
3  CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...  
4  CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...  
5  CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...  
6  CREATE TABLE "Team" (\n\t`id`\tINTEGER PRIMARY...  
7  CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...

In [4]:
countries = pd.read_sql("""SELECT *
                        FROM Country;""", conn)
countries

id         name
0       1      Belgium
1    1729      England
2    4769       France
3    7809      Germany
4   10257        Italy
5   13274  Netherlands
6   15722       Poland
7   17642     Portugal
8   19694     Scotland
9   21518        Spain
10  24558  Switzerland

In [5]:
leagues = pd.read_sql("""SELECT *
                        FROM League
                        JOIN Country ON Country.id = League.country_id;""", conn)
leagues

id  country_id                      name     id         name
0       1           1    Belgium Jupiler League      1      Belgium
1    1729        1729    England Premier League   1729      England
2    4769        4769            France Ligue 1   4769       France
3    7809        7809     Germany 1. Bundesliga   7809      Germany
4   10257       10257             Italy Serie A  10257        Italy
5   13274       13274    Netherlands Eredivisie  13274  Netherlands
6   15722       15722        Poland Ekstraklasa  15722       Poland
7   17642       17642  Portugal Liga ZON Sagres  17642     Portugal
8   19694       19694   Scotland Premier League  19694     Scotland
9   21518       21518           Spain LIGA BBVA  21518        Spain
10  24558       24558  Switzerland Super League  24558  Switzerland

In [6]:
teams = pd.read_sql("""SELECT *
                        FROM Team
                        ORDER BY team_long_name
                        LIMIT 10;""", conn)
teams

id  team_api_id  team_fifa_api_id        team_long_name team_short_name
0  16848         8350                29  1. FC Kaiserslautern             KAI
1  15624         8722                31            1. FC Köln             FCK
2  16239         8165               171        1. FC Nürnberg             NUR
3  16243         9905               169       1. FSV Mainz 05             MAI
4  11817         8576               614            AC Ajaccio             AJA
5  11074       108893            111989      AC Arles-Avignon             ARL
6  49116         6493              1714         AC Bellinzona             BEL
7  26560        10217               650          ADO Den Haag             HAA
8   9537         8583                57            AJ Auxerre             AUX
9   9547         9829                69             AS Monaco             MON

In [7]:
detailed_matches = pd.read_sql("""SELECT Match.id, 
                                        Country.name AS country_name, 
                                        League.name AS league_name, 
                                        season, 
                                        stage, 
                                        date,
                                        HT.team_long_name AS  home_team,
                                        AT.team_long_name AS away_team,
                                        home_team_goal, 
                                        away_team_goal                                        
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                WHERE country_name = 'Spain'
                                ORDER by date
                                LIMIT 10;""", conn)
detailed_matches

id country_name      league_name     season  stage                 date  \
0  21518        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-30 00:00:00   
1  21525        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-30 00:00:00   
2  21519        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-31 00:00:00   
3  21520        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-31 00:00:00   
4  21521        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-31 00:00:00   
5  21522        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-31 00:00:00   
6  21523        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-31 00:00:00   
7  21524        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-31 00:00:00   
8  21526        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-31 00:00:00   
9  21527        Spain  Spain LIGA BBVA  2008/2009      1  2008-08-31 00:00:00   

                   home_team        away_team  home_team_goal  away_team_goal  
0                Valencia CF     RCD Mallorca               3               0  
1               RCD Espanyol  Real Valladolid               1               0  
2                 CA Osasuna    Villarreal CF               1               1  
3  RC Deportivo de La Coruña   Real Madrid CF               2               1  
4                CD Numancia     FC Barcelona               1               0  
5           Racing Santander       Sevilla FC               1               1  
6     Real Sporting de Gijón        Getafe CF               1               2  
7        Real Betis Balompié    RC Recreativo               0               1  
8    Athletic Club de Bilbao       UD Almería               1               3  
9            Atlético Madrid        Málaga CF               4               0

In [8]:
v=leages_by_season = pd.read_sql("""SELECT Country.name AS country_name, 
                                        League.name AS league_name, 
                                        season,
                                        count(distinct stage) AS number_of_stages,
                                        count(distinct HT.team_long_name) AS number_of_teams,
                                        avg(home_team_goal) AS avg_home_team_scors, 
                                        avg(away_team_goal) AS avg_away_team_goals, 
                                        avg(home_team_goal-away_team_goal) AS avg_goal_dif, 
                                        avg(home_team_goal+away_team_goal) AS avg_goals, 
                                        sum(home_team_goal+away_team_goal) AS total_goals                                       
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                WHERE country_name in ('Spain', 'Germany', 'France', 'Italy', 'England')
                                GROUP BY Country.name, League.name, season
                                HAVING count(distinct stage) > 10
                                ORDER BY Country.name, League.name, season DESC
                                ;""", conn)
v

country_name             league_name     season  number_of_stages  \
0       England  England Premier League  2015/2016                38   
1       England  England Premier League  2014/2015                38   
2       England  England Premier League  2013/2014                38   
3       England  England Premier League  2012/2013                38   
4       England  England Premier League  2011/2012                38   
5       England  England Premier League  2010/2011                38   
6       England  England Premier League  2009/2010                38   
7       England  England Premier League  2008/2009                38   
8        France          France Ligue 1  2015/2016                38   
9        France          France Ligue 1  2014/2015                38   
10       France          France Ligue 1  2013/2014                38   
11       France          France Ligue 1  2012/2013                38   
12       France          France Ligue 1  2011/2012                38   
13       France          France Ligue 1  2010/2011                38   
14       France          France Ligue 1  2009/2010                38   
15       France          France Ligue 1  2008/2009                38   
16      Germany   Germany 1. Bundesliga  2015/2016                34   
17      Germany   Germany 1. Bundesliga  2014/2015                34   
18      Germany   Germany 1. Bundesliga  2013/2014                34   
19      Germany   Germany 1. Bundesliga  2012/2013                34   
20      Germany   Germany 1. Bundesliga  2011/2012                34   
21      Germany   Germany 1. Bundesliga  2010/2011                34   
22      Germany   Germany 1. Bundesliga  2009/2010                34   
23      Germany   Germany 1. Bundesliga  2008/2009                34   
24        Italy           Italy Serie A  2015/2016                38   
25        Italy           Italy Serie A  2014/2015                38   
26        Italy           Italy Serie A  2013/2014                38   
27        Italy           Italy Serie A  2012/2013                38   
28        Italy           Italy Serie A  2011/2012                38   
29        Italy           Italy Serie A  2010/2011                38   
30        Italy           Italy Serie A  2009/2010                38   
31        Italy           Italy Serie A  2008/2009                38   
32        Spain         Spain LIGA BBVA  2015/2016                38   
33        Spain         Spain LIGA BBVA  2014/2015                38   
34        Spain         Spain LIGA BBVA  2013/2014                38   
35        Spain         Spain LIGA BBVA  2012/2013                38   
36        Spain         Spain LIGA BBVA  2011/2012                38   
37        Spain         Spain LIGA BBVA  2010/2011                38   
38        Spain         Spain LIGA BBVA  2009/2010                38   
39        Spain         Spain LIGA BBVA  2008/2009                38   

    number_of_teams  avg_home_team_scors  avg_away_team_goals  avg_goal_dif  \
0                20             1.492105             1.207895      0.284211   
1                20             1.473684             1.092105      0.381579   
2                20             1.573684             1.194737      0.378947   
3                20             1.557895             1.239474      0.318421   
4                20             1.589474             1.215789      0.373684   
5                20             1.623684             1.173684      0.450000   
6                20             1.697368             1.073684      0.623684   
7                20             1.400000             1.078947      0.321053   
8                20             1.436842             1.089474      0.347368   
9                20             1.410526             1.081579      0.328947   
10               20             1.415789             1.039474      0.376316   
11               20             1.468421             1.076316      0.392105   
12               20    

In [11]:
players_height = pd.read_sql("""SELECT CASE
                                        WHEN ROUND(height)<165 then 165
                                        WHEN ROUND(height)>195 then 195
                                        ELSE ROUND(height)
                                        END AS calc_height, 
                                        COUNT(height) AS distribution, 
                                        (avg(PA_Grouped.avg_overall_rating)) AS avg_overall_rating,
                                        (avg(PA_Grouped.avg_potential)) AS avg_potential,
                                        AVG(weight) AS avg_weight 
                            FROM PLAYER
                            LEFT JOIN (SELECT Player_Attributes.player_api_id, 
                                        avg(Player_Attributes.overall_rating) AS avg_overall_rating,
                                        avg(Player_Attributes.potential) AS avg_potential  
                                        FROM Player_Attributes
                                        GROUP BY Player_Attributes.player_api_id) 
                                        AS PA_Grouped ON PLAYER.player_api_id = PA_Grouped.player_api_id
                            GROUP BY calc_height
                            ORDER BY calc_height Desc
                                ;""", conn)
players_height

calc_height  distribution  avg_overall_rating  avg_potential  avg_weight
0         195.0           211           67.425619      72.615373  196.464455
1         193.0           470           67.485141      72.459225  188.795745
2         191.0           652           66.997649      71.846159  184.791411
3         188.0          1305           67.094253      72.151949  179.278161
4         185.0          1278           66.928964      71.833475  174.636933
5         183.0          1954           66.634380      71.754555  170.167861
6         180.0          1388           66.419053      71.846394  165.261527
7         178.0          1489           66.367212      71.943339  160.665547
8         175.0          1188           66.805204      72.258774  156.111953
9         173.0           530           66.980272      72.848746  152.824528
10        170.0           403           67.726903      73.379056  147.799007
11        168.0           118           67.500518      73.124182  144.127119
12        165.0            74           67.365543      73.327754  139.459459